In [1]:
!pip install numpy==1.24.4 pandas==1.5.3 openpyxl==3.0.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 75.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [9]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter
import unicodedata


# ExcelファイルをDataFrameに読み込む
df_2022 = pd.read_excel("2022_年間売上表.xlsx", sheet_name=0)
df_2023 = pd.read_excel("2023_年間売上表.xlsx", sheet_name=0)

# データフレームを縦結合
df_all = pd.concat([df_2022, df_2023], ignore_index=True)

#「商品」「売上年」ごとに合計
df_sum = df_all.groupby(["商品", "売上年"], as_index=False)["金額（千円）"].sum()

# 集計結果をExcelに書き込み
out_file = "売上集計表.xlsx"
df_sum.to_excel(out_file, index=False, sheet_name="売上集計")

# openpyxlで表の体裁整備
wb = load_workbook(out_file)
ws = wb["売上集計"]

# (1) ヘッダー行を太字・中央揃え,背景色（薄いグレー #F2F2F2）
header_fill = PatternFill(fill_type="solid", start_color="F2F2F2", end_color="F2F2F2")
header_font = Font(bold=True)
header_align = Alignment(horizontal="center", vertical="center")

for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_align

# 列幅を自動調整（全角文字は2文字分としてカウント）
def get_display_width(text):
    """文字列の表示幅（全角＝2、半角＝1）を返す"""
    width = 0
    for ch in str(text):
        if unicodedata.east_asian_width(ch) in ["F", "W", "A"]:
            width += 2
        else:
            width += 1
    return width

for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        if cell.value is not None:
            max_length = max(max_length, get_display_width(cell.value))
    ws.column_dimensions[col_letter].width = max_length + 2  # 少し余裕を足す

# 保存
wb.save(out_file)